In [2]:
import numpy as np
from dataclasses import dataclass


@dataclass
class Strategy:
    name: str
    base: float
    returns: list[float]


@dataclass
class Asset:
    name: str
    strats: list[Strategy]
    vrange: tuple[float, float]


@dataclass
class Sector:
    name: str
    assets: list[Asset]
    vrange: tuple[float, float]


def gen_fake_data():
    sectors = [
        Sector(
            f"sector_{i}",
            [
                Asset(
                    f"asset_{j}",
                    [
                        Strategy(
                            f"Strat_{i}_{j}_{k}",
                            np.random.uniform(2e5, 5e5),
                            np.random.normal(5e3, 2e4, 20),
                        )
                        for k in range(3)
                    ],
                    (0, None),
                )
                for j in range(10)
            ],
            (0, None),
        )
        for i in range(5)
    ]
    return sectors


fake = gen_fake_data()

In [4]:
# Portfolio Optimization

from gekko import GEKKO


def solve(sectors: list[Sector], delta: float, alpha: float, C: float):
    """Solve the Portfolio Selection Problem.
    This is a non-convex quadratic optimization problem, where risk is counted in the objective function.
    Constraints are all linear, and the objective function is a non-convex quadratic function.
    Can be solved using GEKKO.

    Args:
        sectors (list[Sector]): list of sectors, each sector with a list of assets, each asset with a list of strategies.
        delta (float): Downside-Risk-Aversion parameter.
        alpha (float): Risk(Variance)-Aversion parameter.
        C (float): Total budget constraint.

    Raises:
        ValueError: If the number of days in the returns of the strategies are not the same.

    Returns:
        w[i][j][k]: Weight allocated to strategy k of asset j of sector i.
        By default, the given expected return in input is under weight 1 condition.
        That means, if one asset has weight '2', then you should allocate twice the amount of money to that asset compared to current tested scenario.
    """
    # validate days
    d = len(sectors[0].assets[0].strats[0].returns)
    for s in sectors:
        for a in s.assets:
            for st in a.strats:
                if len(st.returns) != d:
                    raise ValueError(
                        "All strategies must have the same number of days."
                    )

    M = GEKKO(remote=False)
    M.options.IMODE = 3
    M.options.MAX_ITER = 100
    M.options.OTOL = 1e-3
    M.options.RTOL = 1e-3
    DELTA = delta
    ALPHA = alpha
    w = [
        [
            [
                M.Var(name=f"w_{i},{j},{k}", lb=0.0, ub=None)
                for k in range(len(sectors[i].assets[j].strats))
            ]
            for j in range(len(sectors[i].assets))
        ]
        for i in range(len(sectors))
    ]

    # constraints
    constraints = []
    asset_sum = [
        [
            M.sum(
                [
                    w[i][j][k] * sectors[i].assets[j].strats[k].base
                    for k in range(len(sectors[i].assets[j].strats))
                ]
            )
            for j in range(len(sectors[i].assets))
        ]
        for i in range(len(sectors))
    ]
    sector_sum = [
        sum([asset_sum[i][j] for j in range(len(sectors[i].assets))])
        for i in range(len(sectors))
    ]
    total_sum = sum(sector_sum)
    constraints.append(total_sum <= C)
    for i in range(len(sectors)):
        s = sectors[i]
        if s.vrange[0] is not None:
            constraints.append(sector_sum[i] >= s.vrange[0])
        if s.vrange[1] is not None:
            constraints.append(sector_sum[i] <= s.vrange[1])
        for j in range(len(s.assets)):
            a = s.assets[j]
            if a.vrange[0] is not None:
                constraints.append(asset_sum[i][j] >= a.vrange[0])
            if a.vrange[1] is not None:
                constraints.append(asset_sum[i][j] <= a.vrange[1])

    # calculate expected return
    returns = [
        M.sum(
            [
                M.sum(
                    [
                        M.sum(
                            [
                                w[i][j][k] * sectors[i].assets[j].strats[k].returns[t]
                                for k in range(len(sectors[i].assets[j].strats))
                            ]
                        )
                        for j in range(len(sectors[i].assets))
                    ]
                )
                for i in range(len(sectors))
            ]
        )
        for t in range(d)
    ]
    mean = M.sum(returns) / d
    variance = M.sum([(r - mean) ** 2 for r in returns]) / (d - 1)
    U = [M.Var(lb=None, ub=None, name=f"U_{t}") for t in range(d)]
    for i in range(d):
        constraints.append(U[i] <= returns[i])
        constraints.append(U[i] <= 0)
    risk = M.sum([x**2 for x in U]) / (d)
    for cons in constraints:
        M.Equation(cons)
        # M.addCons(cons)
    M.Maximize(sum(returns) - DELTA * risk - ALPHA * variance)
    M.solve(debug=0)
    return [
        [[w[i][j][k].value[0] for k in range(len(w[i][j]))] for j in range(len(w[i]))]
        for i in range(len(sectors))
    ]


res = solve(fake, 1e-3, 5e-4, 1e6)
vres = [
    [
        [res[i][j][k] * fake[i].assets[j].strats[k].base for k in range(len(res[i][j]))]
        for j in range(len(res[i]))
    ]
    for i in range(len(res))
]
d = len(fake[0].assets[0].strats[0].returns)
returns = [
    sum(
        [
            sum(
                [
                    sum(
                        [
                            res[i][j][k] * fake[i].assets[j].strats[k].returns[t]
                            for k in range(len(fake[i].assets[j].strats))
                        ]
                    )
                    for j in range(len(fake[i].assets))
                ]
            )
            for i in range(len(fake))
        ]
    )
    for t in range(d)
]
display(fake)
display(res)
display(vres)
display(returns)
display(sum(returns))

 ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :         1173
   Constants    :            0
   Variables    :         4629
   Intermediates:            0
   Connections  :         5483
   Equations    :         3287
   Residuals    :         3287
 
 Number of state variables:           4629
 Number of total equations: -         4459
 Number of slack variables: -           96
 ---------------------------------------
 Degrees of freedom       :             74
 
 solver            3  not supported
 using default solver: APOPT
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0  2.55362E+14  1.00000E+06
    1 -1.32124E+02  1.94915E+03
    2 

[Sector(name='sector_0', assets=[Asset(name='asset_0', strats=[Strategy(name='Strat_0_0_0', base=302024.52523039805, returns=array([  7094.46602613,  44553.9166221 ,   1141.92821526,  10584.68031911,
        -12794.39229813,    557.89861018,  41761.39514179,  20576.91544453,
            49.09923094,   7402.55936289,   2113.92082339, -35311.16072848,
         15679.40834746,  -3886.42240673,   3909.86657742,  24832.61402621,
         11845.18562703, -11788.61701355,  25873.09750656,   5439.62413113])), Strategy(name='Strat_0_0_1', base=498200.9268488108, returns=array([  -316.54072051,  20567.94606127,   3080.12944598,   8298.93145622,
         15641.54329522, -13609.68263906,   -996.88654433,   3783.07679051,
        -11202.80265887, -23507.36521887, -12477.11792486,  34497.09366185,
        -11461.29771715,   8154.61505739, -16542.73463608, -13254.51068745,
         14338.17396287, -12807.11981423,  28371.96635098,  16739.68944149])), Strategy(name='Strat_0_0_2', base=458169.556363658

[[[0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.14961077889],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0],
  [0.18451690475, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0],
  [0.20163273311, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.75815714878, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[0.24273883307, 0.0, 0.0],
  [0.0, 0.0, 0.088504446835],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.3933670407],
  [0.26739351126, 0.0, 0.0],
  [0.0, 0.0, 0.3410381427]],
 [[0.0, 0.0, 0.0],
  [0.91197669903, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0

[[[0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 57391.27066883948],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0],
  [44714.91800846125, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0],
  [53927.28284854105, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 181434.9148412327, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[94279.52769796542, 0.0, 0.0],
  [0.0, 0.0, 27216.847096949565],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 88286.08083778908],
  [57162.295041109835, 0.0, 0.0],
  [0.0, 0.0, 146428.55533355655]],
 [[0.0, 0.0, 0.0],
  [249158.30761819083, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0

[np.float64(54311.837364631974),
 np.float64(62739.64316279101),
 np.float64(33701.329010666144),
 np.float64(50214.78282742583),
 np.float64(32236.129140291454),
 np.float64(42591.8053440868),
 np.float64(55952.55052213611),
 np.float64(75875.28977939476),
 np.float64(34608.753256489195),
 np.float64(63131.14861424902),
 np.float64(39482.79900043363),
 np.float64(51747.912804572974),
 np.float64(38373.62450595024),
 np.float64(51776.17559720535),
 np.float64(37236.038332417695),
 np.float64(64389.051784278185),
 np.float64(52872.61315421529),
 np.float64(39530.63053929168),
 np.float64(60031.221958269845),
 np.float64(47580.025955946745)]

np.float64(988383.362654744)

In [5]:
def solve_linear(sectors: list[Sector], delta: float, alpha: float, C: float):
    """A slightly modified version of the previous model.
    Risk is presented in the constraints, and the objective function is linear.
    Quadratic-constraint, linear-objective optimization problem. Still can be solved using GEKKO.
    Note that since we can limit the risks directly, it's more intuitive to use this model.

    Args:
        sectors (list[Sector]): list of sectors, each sector with a list of assets, each asset with a list of strategies.
        delta (float): Downside-Risk-Aversion parameter.
        alpha (float): Risk(Variance)-Aversion parameter.
        C (float): Total budget constraint.

    Raises:
        ValueError: If the number of days in the returns of the strategies are not the same.

    Returns:
        w[i][j][k]: Weight allocated to strategy k of asset j of sector i.
        By default, the given expected return in input is under weight 1 condition.
        That means, if one asset has weight '2', then you should allocate twice the amount of money to that asset compared to current tested scenario.
    """

    # validate days
    d = len(sectors[0].assets[0].strats[0].returns)
    for s in sectors:
        for a in s.assets:
            for st in a.strats:
                if len(st.returns) != d:
                    raise ValueError(
                        "All strategies must have the same number of days."
                    )

    M = GEKKO(remote=False)
    M.options.IMODE = 3
    M.options.MAX_ITER = 100
    M.options.OTOL = 1e-3
    M.options.RTOL = 1e-3
    DELTA = delta
    ALPHA = alpha
    w = [
        [
            [
                M.Var(name=f"w_{i},{j},{k}", lb=0.0, ub=None)
                for k in range(len(sectors[i].assets[j].strats))
            ]
            for j in range(len(sectors[i].assets))
        ]
        for i in range(len(sectors))
    ]

    # constraints
    constraints = []
    asset_sum = [
        [
            M.sum(
                [
                    w[i][j][k] * sectors[i].assets[j].strats[k].base
                    for k in range(len(sectors[i].assets[j].strats))
                ]
            )
            for j in range(len(sectors[i].assets))
        ]
        for i in range(len(sectors))
    ]
    sector_sum = [
        sum([asset_sum[i][j] for j in range(len(sectors[i].assets))])
        for i in range(len(sectors))
    ]
    total_sum = sum(sector_sum)
    constraints.append(total_sum <= C)
    for i in range(len(sectors)):
        s = sectors[i]
        if s.vrange[0] is not None:
            constraints.append(sector_sum[i] >= s.vrange[0])
        if s.vrange[1] is not None:
            constraints.append(sector_sum[i] <= s.vrange[1])
        for j in range(len(s.assets)):
            a = s.assets[j]
            if a.vrange[0] is not None:
                constraints.append(asset_sum[i][j] >= a.vrange[0])
            if a.vrange[1] is not None:
                constraints.append(asset_sum[i][j] <= a.vrange[1])

    # calculate expected return
    returns = [
        M.sum(
            [
                M.sum(
                    [
                        M.sum(
                            [
                                w[i][j][k] * sectors[i].assets[j].strats[k].returns[t]
                                for k in range(len(sectors[i].assets[j].strats))
                            ]
                        )
                        for j in range(len(sectors[i].assets))
                    ]
                )
                for i in range(len(sectors))
            ]
        )
        for t in range(d)
    ]
    mean = M.sum(returns) / d
    variance = M.sum([(r - mean) ** 2 for r in returns]) / (d - 1)
    std = M.sqrt(variance)
    U = [M.Var(lb=None, ub=None, name=f"U_{t}") for t in range(d)]
    for i in range(d):
        constraints.append(U[i] <= returns[i] - mean + 3 * std)
        constraints.append(U[i] <= 0)
    risk = M.sum([x**2 for x in U]) / (d)
    for cons in constraints:
        M.Equation(cons)
        # M.addCons(cons)
    M.Equation(risk <= DELTA)
    M.Equation(variance <= ALPHA)
    M.Maximize(sum(returns))
    M.solve(debug=0)
    return [
        [[w[i][j][k].value[0] for k in range(len(w[i][j]))] for j in range(len(w[i]))]
        for i in range(len(sectors))
    ]


res = solve_linear(fake, 1e6, 5e7, 1e6)
vres = [
    [
        [res[i][j][k] * fake[i].assets[j].strats[k].base for k in range(len(res[i][j]))]
        for j in range(len(res[i]))
    ]
    for i in range(len(res))
]
d = len(fake[0].assets[0].strats[0].returns)
returns = [
    sum(
        [
            sum(
                [
                    sum(
                        [
                            res[i][j][k] * fake[i].assets[j].strats[k].returns[t]
                            for k in range(len(fake[i].assets[j].strats))
                        ]
                    )
                    for j in range(len(fake[i].assets))
                ]
            )
            for i in range(len(fake))
        ]
    )
    for t in range(d)
]
display(fake)
display(res)
display(vres)
display(returns)
display(sum(returns))
print(
    "mean:",
    sum(returns) / d,
    "variance:",
    sum([(r - sum(returns) / d) ** 2 for r in returns]) / (d - 1),
)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :         1173
   Constants    :            0
   Variables    :         4631
   Intermediates:            0
   Connections  :         5483
   Equations    :         3289
   Residuals    :         3289
 
 Number of state variables:           4631
 Number of total equations: -         4461
 Number of slack variables: -           98
 ---------------------------------------
 Degrees of freedom       :             72
 
 solver            3  not supported
 using default solver: APOPT
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0  7.31327E+16  5.00000E+07
    1  2.10046E+08  5.49316E+01
    2 

[Sector(name='sector_0', assets=[Asset(name='asset_0', strats=[Strategy(name='Strat_0_0_0', base=302024.52523039805, returns=array([  7094.46602613,  44553.9166221 ,   1141.92821526,  10584.68031911,
        -12794.39229813,    557.89861018,  41761.39514179,  20576.91544453,
            49.09923094,   7402.55936289,   2113.92082339, -35311.16072848,
         15679.40834746,  -3886.42240673,   3909.86657742,  24832.61402621,
         11845.18562703, -11788.61701355,  25873.09750656,   5439.62413113])), Strategy(name='Strat_0_0_1', base=498200.9268488108, returns=array([  -316.54072051,  20567.94606127,   3080.12944598,   8298.93145622,
         15641.54329522, -13609.68263906,   -996.88654433,   3783.07679051,
        -11202.80265887, -23507.36521887, -12477.11792486,  34497.09366185,
        -11461.29771715,   8154.61505739, -16542.73463608, -13254.51068745,
         14338.17396287, -12807.11981423,  28371.96635098,  16739.68944149])), Strategy(name='Strat_0_0_2', base=458169.556363658

[[[0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.2109786137],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0],
  [0.18938708107, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0],
  [0.18703656737, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.60600597848, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.039206064632, 0.0]],
 [[0.18402405172, 0.0, 0.0],
  [0.0, 0.0, 0.17265184868],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.37312550545],
  [0.1943648659, 0.0, 0.0],
  [0.0, 0.0, 0.39973020732]],
 [[0.0, 0.0, 0.10540669937],
  [0.72788141603, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [

[[[0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 80932.20832100454],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0],
  [45895.13255373884, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0],
  [50023.49428095888, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 145023.5525388443, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 9981.075759205363, 0.0]],
 [[71474.76348056896, 0.0, 0.0],
  [0.0, 0.0, 53093.81770714542],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 83743.13332957285],
  [41550.52887352382, 0.0, 0.0],
  [0.0, 0.0, 171628.65220193053]],
 [[0.0, 0.0, 47791.43103336995],
  [198862.2099201256, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.

[np.float64(49211.913848959965),
 np.float64(58054.69616430528),
 np.float64(37496.87519141069),
 np.float64(47698.30007093359),
 np.float64(36001.65342079979),
 np.float64(37345.098334411196),
 np.float64(51000.85953333286),
 np.float64(57685.96712108972),
 np.float64(36490.823044263656),
 np.float64(53575.16399174202),
 np.float64(39267.404637857646),
 np.float64(50012.64777542197),
 np.float64(41519.013872513766),
 np.float64(51233.211589450235),
 np.float64(41562.00105675199),
 np.float64(51276.1054557475),
 np.float64(44887.22885077882),
 np.float64(37968.54737844947),
 np.float64(49399.99124631335),
 np.float64(44630.42773842272)]

np.float64(916317.9303229561)

mean: 45815.896516147805 variance: 50000000.27520156
